In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [ ]:
class CustomDataset(Dataset):
	def __init__(self):
		pass

	def __getitem__(self, idx):
		pass

	def __len__(self):
		pass

In [ ]:
class DataHandler:
	def __init__(self, run_config):
		raise Exception("Must implement!")
		self._training_dataset = None
		self._validation_dataset = None
		self._run_config = run_config

		self.load_datasets()
		
	def load_datasets(self):
		self._training_dataset = CustomDataset(...)
		self._validation_dataset = CustomDataset(...)

	def get_data_loaders(self) -> Tuple[DataLoader]:
		return (
			DataLoader(self._training_dataset, batch_size=self._run_config.batch_size, shuffle=True, num_workers=self._run_config.workers, pin_memory=True), 
			DataLoader(self._validation_dataset, batch_size=self._run_config.batch_size, shuffle=True, num_workers=self._run_config.workers, pin_memory=True)
		)

	def get_datasets(self) -> Tuple[Dataset]:
		return self._training_dataset, self._validation_dataset

	def get_datasets_sizes(self) -> Tuple[int]:
		return len(self._training_dataset), len(self._validation_dataset)

In [ ]:
model = ...
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
loss_criterion = ...

In [ ]:
for epoch in range(EPOCHS):
	print('Epoch {}/{}'.format(epoch, run.num_epochs))
	print('-' * 10)

	########### Training step ###########
	model.train()
	training_loss = []
	running_loss = 0.0
	running_corrects = 0
			
	for i, data in enumerate(tqdm(train_loader, desc=f"Epoch [{epoch + 1}] progress")):

		x_batch, label_batch = data
		x_batch, label_batch = x_batch.to(device), label_batch.to(device)

		optimizer.zero_grad()
		outputs = model(x_batch)
		_, preds = torch.max(outputs, 1)

		loss = loss_criterion(outputs, label_batch)

		loss.backward()
		optimizer.step()
				
		# statistics
		running_loss += loss.item() * x_batch.size(0)
		running_corrects += torch.sum(preds == label_batch.detach())
		training_loss.append(loss.item())

	epoch_loss = running_loss / training_dataset_size
	epoch_acc = running_corrects.double() / training_dataset_size

	# tensorboard logging
	writer.add_scalar("Loss/train", epoch_loss, epoch)
	writer.add_scalar("Accuracy/train", epoch_acc, epoch)

	print('Training step => Loss: {:.4f} Acc: {:.4f}'.format(
		epoch_loss, epoch_acc
	))

	scheduler.step(epoch_acc)


	########### Validation step ###########
	model.eval()
	validation_loss = []
	running_loss = 0.0
	running_corrects = 0

	for i, data in enumerate(validation_loader):
		with torch.no_grad():
			x_batch, label_batch = data
			x_batch, label_batch = x_batch.to(device), label_batch.to(device)

			outputs = model(x_batch)
			_, preds = torch.max(outputs, 1)
			loss = loss_criterion(...)

			running_loss += loss.item() * x_batch.size(0)
			running_corrects += torch.sum(preds == label_batch.detach())
			validation_loss.append(loss.item())
			
	epoch_loss = running_loss / validation_dataset_size
	epoch_acc = running_corrects.double() / validation_dataset_size

	# tensorboard logging
	writer.add_scalar("Loss/validation", epoch_loss, epoch)
	writer.add_scalar("Accuracy/validation", epoch_acc, epoch)

	print('Evaluation step => Loss: {:.4f} Acc: {:.4f}'.format(
		epoch_loss, epoch_acc
	))

	#Save the best model based on accuracy
	if epoch_acc > best_acc:
		best_acc = epoch_acc
		best_config = f"{run}"
		best_model_wts = copy.deepcopy(model.state_dict())

	#Checkpoint
	torch.save({
		"epoch": epoch,
		"model_state_dict": model.state_dict(),
		"optimizer_state_dict": optimizer.state_dict()
	}, "./checkpoints/ckp.pt")



time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
	time_elapsed // 60, time_elapsed % 60
))
print('Best (so far) validation Acc: {:4f}'.format(best_acc))

print('-' * 10)
print('### Final results ###\n')
print('Best validation Acc: {:4f}'.format(best_acc))

model.load_state_dict(best_model_wts)